In [339]:
import numpy as np
import pandas as pd
import scipy.stats as stat
import scipy.io as scipio
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [340]:
data_train = pd.DataFrame(pd.read_csv("data_train.csv")).set_index("visit_date",drop=True)
data_test = pd.DataFrame(pd.read_csv("data_test.csv")).set_index("visit_date",drop=True)
data = pd.concat([data_train, data_test],0)
                         

In [341]:
area_names = data_train.columns.values[29:]
genre_names = data_train.columns.values[15:29]
weather_measurements = data_train.columns.values[[8,9,11,12]]

In [342]:
data_test.index = pd.Series(data_test.index.values).apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))
data_train.index = pd.Series(data_train.index.values).apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

In [343]:
data.index = pd.Series(data.index.values).apply(lambda x: datetime.strptime(x, "%Y-%m-%d"))

In [344]:
data_train[weather_measurements] = scaler.fit_transform(data_train[weather_measurements].diff().fillna(method="backfill"))
data_test[weather_measurements] = scaler.fit_transform(data_test[weather_measurements].diff().fillna(method="backfill"))
data[weather_measurements] = scaler.fit_transform(data[weather_measurements].diff().fillna(method="backfill"))

In [345]:
## Taking out February as the validation set from the training data
validation_ind = pd.Series(data_train.index.values).apply(lambda x: int((x >= datetime.strptime("2017-02-01","%Y-%m-%d")) & (x <= datetime.strptime("2017-02-28","%Y-%m-%d"))))
data_train["validation_ind"] = list(validation_ind)


In [346]:
data_train_train = data_train[data_train["validation_ind"]==0]
data_train_validation = data_train[data_train["validation_ind"]==1]

In [347]:
list(area_names).extend(list(genre_names))

In [348]:
genre_names = list(genre_names)
non_arima_cols = list(area_names)
non_arima_cols.extend(genre_names)

In [53]:
# non_arima_cols

In [62]:
# data_train_train.columns.values

In [349]:
arima_cols = ["holiday_flg", "low_temperature1", "high_temperature1", "avg_temperature1","hours_sunlight1",\
             "visitors", "Monday", "Tuesday","Wednesday", "Thursday","Friday","Saturday","Sunday", "air_store_id"]
# data_train_train[arima_cols]

In [350]:
data_train_train.to_csv("training_wo_valid.csv")
data_train_validation.to_csv("training_valid.csv")

## ARIMA

In [351]:
data_train_train = data_train_train.drop(data_train_train[data_train_train["air_store_id"]=="air_28dbe91c4c9656be"],0)
data_train_train = data_train_train.drop(data_train_train[data_train_train["air_store_id"]=="air_8110d68cc869b85e"], 0)



In [352]:
data_train_train[data_train_train["air_store_id"]=="air_28dbe91c4c9656be"]

,Unnamed: 0,air_store_id,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday,avg_temperature1,...,Ōsaka-fu Sakai-shi Minamikawaramachi,Ōsaka-fu Suita-shi Izumichō,Ōsaka-fu Ōsaka-shi Fuminosato,Ōsaka-fu Ōsaka-shi Kyōmachibori,Ōsaka-fu Ōsaka-shi Kyūtarōmachi,Ōsaka-fu Ōsaka-shi Nakanochō,Ōsaka-fu Ōsaka-shi Nanbasennichimae,Ōsaka-fu Ōsaka-shi Shinmachi,Ōsaka-fu Ōsaka-shi Ōgimachi,validation_ind
2017-01-30,157032,air_28dbe91c4c9656be,0,1,0,0,0,0,0.0,-0.848822,...,0,0,0,0,0,0,0,0,0,0
2017-01-31,157603,air_28dbe91c4c9656be,0,0,0,0,0,1,0.0,-0.636620,...,0,0,0,0,0,0,0,0,0,0


In [353]:
rest_data_dict = {}

for restaurant in \
list(set(data["air_store_id"])):
    rest_data_dict[restaurant] = data[data["air_store_id"]==restaurant]
    
arima_rest_data_dict = {}
arima_data = data[arima_cols]
for restaurant in \
list(set(data["air_store_id"])):
    arima_rest_data_dict[restaurant] = arima_data[arima_data["air_store_id"]==restaurant]

In [354]:
# arima_rest_data_dict["air_83db5aff8f50478e"] ## check

In [355]:
train_dict = {}
validation_dict = {}
test_dict = {}

arima_train_dict = {}
arima_val_dict = {}
arima_test_dict = {}

for key in rest_data_dict.keys():
    ## Train
    if len(data_train_train[data_train_train["air_store_id"]==key]) >= 60:
        train_dict[key] = data_train_train[data_train_train["air_store_id"]==key]
    ## Validation
    validation_dict[key] = data_train_validation[data_train_validation["air_store_id"]==key]
    ## Test
    test_dict[str(key)] = rest_data_dict[key][rest_data_dict[key]["train_ind"]==0]
    ## Train ARIMA
    if len(data_train_train[data_train_train["air_store_id"]==key]) >= 20:
        arima_train_dict[key] = data_train_train[data_train_train["air_store_id"]==key][arima_cols]
    ## Validation ARIMA
    arima_val_dict[key] = data_train_validation[data_train_validation["air_store_id"]==key][arima_cols]
    ## Test ARIMA
    arima_test_dict[key] = rest_data_dict[key][rest_data_dict[key]["train_ind"]==0][arima_cols]
    

In [356]:
## rest_data_dict = {"restaurant_ids": DFs}
## train_dict = {"restaurant_ids": train_DFs}
## validation_dict = {"restaurant_ids": validation_DFs}
## test_dict = {"restaurant_ids": test_DFs}

In [357]:
from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.preprocessing import StandardScaler
from datetime import datetime
from statsmodels.tsa.arima_model import ARMAResults

In [358]:
train_exog_cols = np.where((data_train_train[arima_cols].columns.values != "visitors") & (data_train_train[arima_cols].columns.values != "train_ind") &\
         (data_train_train[arima_cols].columns.values != "visit_date") & (data_train_train[arima_cols].columns.values != "air_store_id") \
         & (data_train_train[arima_cols].columns.values != "validation_ind"))

endo = {}
exog = {}
arima_dates = {}

for restaurant in list(set(list(arima_train_dict.keys()))):
    endo[restaurant] = pd.DataFrame(arima_train_dict[restaurant]["visitors"], \
                                    index=arima_train_dict[restaurant].index)

    exog[restaurant] = arima_train_dict[restaurant][arima_train_dict[restaurant].\
                                              columns[train_exog_cols]]
    
    arima_dates[restaurant] = endo[restaurant].index

In [359]:
## same for validation
valid_exog_cols = np.where((data_train_validation[arima_cols].columns.values != "visitors") & (data_train_validation[arima_cols].columns.values != "train_ind") &\
         (data_train_validation[arima_cols].columns.values != "visit_date") & (data_train_validation[arima_cols].columns.values != "air_store_id") \
         & (data_train_validation[arima_cols].columns.values != "validation_ind"))

endo_valid = {}
exog_valid = {}
arima_dates_valid = {}

for restaurant in list(set(list(arima_val_dict.keys()))):
    endo_valid[restaurant] = pd.DataFrame(arima_val_dict[restaurant]["visitors"], \
                                    index=arima_val_dict[restaurant].index)

    exog_valid[restaurant] = arima_val_dict[restaurant][arima_val_dict[restaurant].\
                                              columns[valid_exog_cols]]
    
    arima_dates_valid[restaurant] = arima_val_dict[restaurant].index

In [360]:
model_fit_try = ARIMA(endog=np.array(endo["air_83db5aff8f50478e"]), order=(3,1,0),\
                                    exog=np.array(exog["air_83db5aff8f50478e"]), dates=arima_dates["air_83db5aff8f50478e"], freq="D") 

In [300]:
model_fit_try.score

<bound method ARMA.score of <statsmodels.tsa.arima_model.ARIMA object at 0x1a3ea72240>>

In [363]:
model_fit_fit = model_fit_try.fit(start_params=None, trend="nc", method="css-mle", maxiter=1000, transparams=True, disp=0)



In [364]:
model_fit_fit.mle_retvals

{'converged': True,
 'fcalls': 5664,
 'fopt': 2.933402439587428,
 'gopt': array([ -8.88178420e-08,   5.99520433e-06,   8.12683254e-06,
         -2.66453526e-07,   4.44089210e-07,  -3.10862447e-07,
          1.33226763e-07,   0.00000000e+00,   4.44089210e-08,
          1.77635684e-07,  -1.33226763e-07,  -2.22044605e-07,
         -1.02140518e-06,  -5.32907052e-07,  -2.66453526e-07]),
 'warnflag': 0}

In [365]:
len(exog["air_83db5aff8f50478e"].columns.values)

12

In [366]:
model_fit_fit.mle_retvals

{'converged': True,
 'fcalls': 5664,
 'fopt': 2.933402439587428,
 'gopt': array([ -8.88178420e-08,   5.99520433e-06,   8.12683254e-06,
         -2.66453526e-07,   4.44089210e-07,  -3.10862447e-07,
          1.33226763e-07,   0.00000000e+00,   4.44089210e-08,
          1.77635684e-07,  -1.33226763e-07,  -2.22044605e-07,
         -1.02140518e-06,  -5.32907052e-07,  -2.66453526e-07]),
 'warnflag': 0}

In [367]:
model_fit_fit.params

array([ -3.25424950e+03,  -1.13012827e-01,   1.16481961e-01,
         2.62377079e-01,  -2.28305550e-01,  -6.10079294e-01,
         1.02797560e+00,   1.48088376e-10,   1.68498659e+00,
         5.40497541e-01,   7.01794285e-01,  -1.14451387e+00,
        -9.22867477e-01,  -6.06768921e-01,  -3.60538818e-01])

In [368]:
model_fit_fit.summary()

LinAlgError: Singular matrix

In [104]:
len(exog["air_83db5aff8f50478e"])

177

In [109]:
len(exog_valid["air_83db5aff8f50478e"])

23

In [369]:
## predict for each restaurant id
model_fit_fit.predict(len(exog["air_83db5aff8f50478e"]), len(exog["air_83db5aff8f50478e"])+len(exog_valid["air_83db5aff8f50478e"])-1, exog=exog_valid["air_83db5aff8f50478e"])


array([ 6.94431265, -0.53448508,  2.69572722, -5.59905816,  2.20950005,
       -0.29487594, -0.20307438, -0.10765368,  1.0181342 ,  0.316366  ,
       -0.37648286,  0.60772874,  0.30826647,  1.65403448,  1.31162541,
        0.13182807,  0.52914987, -0.26389208,  1.41875446,  0.10111037,
        0.53494233, -1.30344361,  0.42510741])

In [371]:
def average_percentage_error(pred_array, true_array):
    n = len(true_array)
    percent_error = []
    for i in range(n):
        pct_error = np.absolute(np.subtract(pred_array, true_array))[i]/true_array[i]
        percent_error.append(pct_error)
        
    avg_percent_error = np.mean(percent_error)
    return avg_percent_error

In [130]:
data_train_train = data_train_train.drop(data_train_train[data_train_train["air_store_id"]=="air_28dbe91c4c9656be"])

In [132]:
data_train_train = data_train_train.drop(data_train_train[data_train_train["air_store_id"]=="air_8110d68cc869b85e"])

In [133]:
# for restaurant in list(set(data_train_train["air_store_id"])):
#     print (restaurant + " " + str(len(endo[restaurant])))
    
    ## air_28dbe91c4c9656be
    ## air_8110d68cc869b85e

In [227]:
arima_models_byrest = {}

## for all unique restaurants
for restaurant in list(set(list(arima_train_dict.keys()))):
    ## config the ARIMA model that corresponds to the restaurant,
    ## write it to a dict that has rest_ids as keys
    arima_models_byrest[restaurant] = ARIMA(endog=endo[restaurant], order=(1,2,1),\
                                    exog=exog[restaurant], dates=arima_dates[restaurant], \
                                          freq="D")          

## fitting
arima_fitted_models_byrest = {}
## for all unique restaurants
for restaurant in list(set(list(arima_train_dict.keys()))):
    ## fit the ARIMA model that corresponds to the restaurant 
    ## (from the previous dict),
    ## write it to a dict that has rest_ids as keys
    arima_fitted_models_byrest[restaurant] = arima_models_byrest[restaurant].\
    fit(start_params=None, trend="nc", \
                      method="mle", maxiter=50, transparams=True, disp=0)

preds_by_rest = {}
## predictions for each restaurant
for restaurant in list(set(list(arima_fitted_models_byrest.keys()))):
    ## get the preds of the ARIMA model that corresponds to the restaurant 
    ## (from the previous dict),
    ## write it to a dict that has rest_ids as keys
    preds_by_rest[restaurant] = arima_fitted_models_byrest[restaurant].\
    predict(len(exog[restaurant]), len(exog[restaurant])+len(exog_valid[restaurant])-1, \
            exog=exog_valid[restaurant])
            ## start index ## end index ## exogeneous_validation
        
## this is indifferent among restaurants
## the coefs of all restaurants will have the same weight
## we are looking for a better way of doing this
objective_rest = {}
for restaurant in list(set(list(preds_by_rest.keys()))):
    ## normalized_arima_loss(pred_array, true_array)
    ## takes the prediction of the corresponding restaurant from the 
    ## previous dictionary
    ## true_array is the validation array
    objective_rest[restaurant] = normalized_arima_loss(preds_by_rest[restaurant], \
                                                      endo_valid[restaurant]["visitors"])
    
## append all the losses corresponding to
## each restaurant's model to a list
loss_to_avr = []
for key in list(objective_rest.keys()):
    loss_to_avr.append(objective_rest[key])

## average the losses
np.mean(loss_to_avr)

/Users/derin/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/derin/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/Users/derin/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


ValueError: The computed initial MA coefficients are not invertible
You should induce invertibility, choose a different model order, or you can
pass your own start_params.

In [328]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

In [372]:
arima_models_byrest = {}

## for all unique restaurants
for restaurant in list(set(list(arima_train_dict.keys()))):
    ## config the ARIMA model that corresponds to the restaurant,
    ## write it to a dict that has rest_ids as keys
    arima_models_byrest[restaurant] = ARIMA(endog=endo[restaurant], order=(3,1,0),\
                                    exog=exog[restaurant], dates=arima_dates[restaurant], \
                                          freq="D")      

In [ ]:
## fitting
arima_fitted_models_byrest = {}
## for all unique restaurants
for restaurant in list(set(list(arima_train_dict.keys()))):
    ## fit the ARIMA model that corresponds to the restaurant 
    ## (from the previous dict),
    ## write it to a dict that has rest_ids as keys
    
    ## change start_params length with order
    arima_fitted_models_byrest[restaurant] = arima_models_byrest[restaurant].\
    fit(start_params=None, trend="nc", \
                      tol=1e-9,method="css-mle", maxiter=1000, transparams=True, disp=1)

/Users/derin/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:473: HessianInversionWarning: Inverting hessian failed, no bse or cov_params available
  'available', HessianInversionWarning)
/Users/derin/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/derin/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/derin/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)
/Users/derin/anaconda/lib/python3.6/site-packages/statsmodels/base/model.py:496: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Ch

In [ ]:
preds_by_rest = {}
## predictions for each restaurant
for restaurant in list(set(list(arima_fitted_models_byrest.keys()))):
    ## get the preds of the ARIMA model that corresponds to the restaurant 
    ## (from the previous dict),
    ## write it to a dict that has rest_ids as keys
    if len(exog_valid[restaurant]) > 0:
        preds_by_rest[restaurant] = arima_fitted_models_byrest[restaurant].\
        predict(len(exog[restaurant]), len(exog[restaurant])+len(exog_valid[restaurant])-1, \
            exog=exog_valid[restaurant])
            ## start index ## end index ## exogeneous_validation

In [ ]:
## this is indifferent among restaurants
## the coefs of all restaurants will have the same weight
## we are looking for a better way of doing this
objective_rest = {}
for restaurant in list(set(list(preds_by_rest.keys()))):
    ## normalized_arima_loss(pred_array, true_array)
    ## takes the prediction of the corresponding restaurant from the 
    ## previous dictionary
    ## true_array is the validation array
    objective_rest[restaurant] = average_percentage_error(preds_by_rest[restaurant], \
                                                      scaler.fit_transform(endo_valid[restaurant]["visitors"]))

In [ ]:
objective_rest

In [ ]:
import math
loss_to_avr = []
for key in list(objective_rest.keys()):
    loss_to_avr.append(objective_rest[key])

loss_to_avr = [value for value in loss_to_avr if not math.isnan(value)]
np.mean(loss_to_avr)
# loss_to_avr

In [316]:
loss_to_avr

[0.8050284016773438,
 0.91634795328895402,
 0.86737874624825451,
 1.0122146448686107,
 1.2818863288001141,
 1.0790896418101632,
 1.2342656934642928,
 0.97159200755505115,
 0.94974622853086832,
 1.8173067392507178,
 1.3302228534981579,
 1.2037542513534212,
 1.1276268191707386,
 1.838303831676823,
 4.6616683034812523,
 1.1799030124065744,
 1.3625339320383691,
 1.0239410124980228,
 1.0508336314065887,
 1.188290211928013,
 0.74561506461926153,
 0.77348805157319711,
 1.013407903362356,
 1.1376010975950597,
 1.3103480477801108,
 1.5907351013701359,
 1.3835630211416792,
 1.9083373537571948,
 1.1557470099695997,
 1.1000483092253086,
 1.1232378113520269,
 0.849823445230529,
 1.4122499204970431,
 0.91576105398386642,
 0.84926017755398975,
 1.1411035221192132,
 1.0826992649690697,
 1.0920591235344219,
 0.97020200203017148,
 2.3052452712370597,
 2.4485378225389338,
 1.0945982009566593,
 0.84945797149470792,
 1.0563293666328415,
 1.5481223677325595,
 1.1848141916503743,
 0.74798167520622538,
 1.260

In [158]:
len(list(arima_models_byrest.keys()))

660

In [ ]:
class classARIMA(BaseEstimator, RegressorMixin):
    """ Regularized Logistic Regression"""
    
    def __init__(self, order=(1,0,3),rest_data_dict, train_dict,\
                test_dict, l2param=1, models_dict, validation_dict):
        if len(order) < 0:
            raise ValueError('You should have at least 3 values for order')
        self.order = order
        self.rest_data_dict = rest_data_dict
        self.train_dict = train_dict
        self.validation_dict = validation_dict
        self.test_dict = test_dict
        self.l2param = l2param
        self.models_dict = models_dict
        
    def fit(self, theta):
        order = self.order
        train_dict = self.train_dict
        test_dict = self.test_dict
        num_train_rests = len(list(train_dict.keys())) 
        num_valid_rests = len(list(validation_dict.keys())) 
        num_test_rests = len(list(test_dict.keys()))
        
        num_models = len(list(models_dict.keys()))
        
        ## endo - exog ne yapacagina karar ver 
        train_ready_to_fit = [ARIMA(endog=arima_visitors_endo, order=(1,0,3), \
                              exog=, \
                              dates=arima_dates, freq="D")\
                        for key in list(models_dict.keys())]
        
        valid_ready_to_fit = [ARIMA(endog=endo_valid, order=(1,0,3), \
                              exog=
                              dates=arima_dates, freq="D")\
                        for key in list(models_dict.keys())]
        
        fitted_model_array = np.array([model.fit(start_params=None, trend="nc",\
                                                 method="mle", maxiter=10, transparams=True, disp=0)\
                                      for model in ])
        
        def weighted_objective(theta):
            
            weighted_objective = (1/num_models) 
            return weighted_objective   
        
        self.f_objective = f_objective
        theta_0 = np.zeros(num_models)
        self.theta_ = minimize(f_objective, theta_0).x 
        return self

    def predict(self, X):
        try:
            getattr(self, "theta_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        return np.dot(X, self.theta_)

    def score(self, X, y):
        # Average log loss
        try:
            getattr(self, "theta_")
        except AttributeError:
            raise RuntimeError("You must train classifer before predicting data!")
        ## log-likelihood - not negative
        theta = np.array(self.theta_).reshape(-1)
        y = map_y(np.array(y), method="erm")
        s = np.dot(X, theta)
#         l2param = self.l2param
        h = - logreg_logsumexp(np.dot(np.transpose(y), s))
        log_likelihood = h
        return log_likelihood
    